In [52]:
#TODO: ensure response column is "-1"

# This file contains code to train dropout networks on the UCI datasets using the following algorithm:
# 1. Create 20 random splits of the training-test dataset.
# 2. For each split:
# 3.   Create a validation (val) set taking 20% of the training set.
# 4.   Get best hyperparameters: dropout_rate and tau by training on (train-val) set and testing on val set.
# 5.   Train a network on the entire training set with the best pair of hyperparameters.
# 6.   Get the performance (MC RMSE and log-likelihood) on the test set.
# 7. Report the averaged performance (Monte Carlo RMSE and log-likelihood) on all 20 splits.

from sklearn.model_selection import KFold, train_test_split
from web_dataloader import web_dataloader
from test_mc_dropout import test_mc_dropout
from test_ngboost import test_ngboost

seed= 42

dataset_name = "housing"
model = "vogn"
n_folds = 2
alpha = .05






# Load Dataset
dataset = web_dataloader[dataset_name]() #[args.dataset]()
X, y = dataset.iloc[:, :-1].values, dataset.iloc[:, -1].values

# Get row indices for the K different dataset splits
folds = KFold(n_splits=n_folds, shuffle=False, random_state=seed).split(X)

# # Log results in .txt file
# with open("results_"+model+"/"+dataset_name+".csv", "w") as myfile:
#     myfile.write("coverage,avg_range,test_ll\n")

# For each dataset split get train, validation, testing
for fold_count, (train_index, test_index) in enumerate(folds):
    print(f"\nStarting fold {fold_count+1} for dataset {dataset_name}.\n")

    X_trainall, X_test = X[train_index], X[test_index]
    y_trainall, y_test = y[train_index], y[test_index]

    X_train, X_val, y_train, y_val = train_test_split(
        X_trainall, y_trainall, test_size=0.2
    )       

    # Call testing function
#     coverage, avg_range, test_ll=\
#         test_ngboost(X_train, y_train, X_val, y_val, X_test, y_test, alpha)

#     # Write result
#     with open("results_"+model+"/"+dataset_name+".csv", "a") as myfile:
#         myfile.write(repr(coverage)+","+repr(avg_range)+","+repr(test_ll)+"\n")


Starting fold 1 for dataset housing.


Starting fold 2 for dataset housing.



### VOGN

In [53]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torchsso
from perceptrons import MLP
import torch.nn.functional as F

# Check GPU availability
device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')

X_train= torch.from_numpy(X_train).type(torch.float)
y_train= torch.from_numpy(y_train).type(torch.float)

# Prepare data
train_dataset= TensorDataset(
    X_train,
    y_train
)

X_val = torch.from_numpy(X_val).type(torch.float)
y_val = torch.from_numpy(y_val).type(torch.float)

X_test = torch.from_numpy(X_test).type(torch.float)
y_test = torch.from_numpy(y_test).type(torch.float)

train_loader= DataLoader(train_dataset, batch_size=25)

In [55]:
# Model parameters
model_params = {'hidden_sizes': [50,50,50],
                'act_func': "relu",
                'prior_prec': 1.0}

# Training parameters
train_params = {'num_epochs': 40,
                'batch_size': None,
                'train_mc_samples': None,
                'eval_mc_samples': 100,
                'seed': 123}

# Optimizer parameters
optim_params = {'learning_rate': 0.001,
                'betas': (0.9, 0.999),
                'prec_init': 1.0}


epochs=10

In [75]:
from vadam.optimizers import VOGN
import vadam.metrics as metrics

# Normalize X        
X_means = torch.mean(X_train, dim=0)
X_stds = torch.std(X_train, dim=0)
X_stds[X_stds == 0] = 1
            
# Normalize y
y_mean = torch.mean(y_train)
y_std = torch.std(y_train)
if y_std==0:
    y_std = 1

# Initialize model
vogn_model= MLP(input_size=X_train.shape[1], 
                output_size=1, 
                hidden_sizes=[50,50,50], 
                act_func='relu')

# Initialize optimizer
vogn_optimizer = VOGN(vogn_model.parameters(),
                      lr = 1e-3,
                      beta = .99,
                      prior_prec = 1.,
                      prec_init = 1.,
                      num_samples = 10,
                      train_set_size = X_train.shape[0])

# Train model
for epoch in range(epochs):

    # Set model in training mode
    vogn_model.train(True)

    # Initialize batch objective accumulator
    batch_objective = []

    for i, (X, y) in enumerate(train_loader):

        # Normalize x and y
        X= (X- X_means)/ X_stds
        y= (y-y_mean)/ y_std

        # Update parameters
        def closure():
            vogn_optimizer.zero_grad()
            output = vogn_model(X)
            loss = metrics.avneg_loglik_gaussian(output, y, tau = 1.)
            loss.backward()
            return loss, output
        loss, _ = vogn_optimizer.step(closure)

ValueError: not enough values to unpack (expected 4, got 2)

In [69]:
y_train[1]

tensor(29.9000)

In [74]:
output = vogn_model(X_train[1,])
metrics.avneg_loglik_gaussian(output.squeeze(), y, tau = 1.)

tensor(3.3068e+09, grad_fn=<NegBackward>)

In [ ]:
        # Store batch objective
        batch_objective.append(loss.detach().cpu().item())
        print(f'Train Epoch: {epoch+1}\tLoss(VOGN): {loss:.6f}')

#     # Compute and store average objective from last epoch
#     self.objective_history[split].append(np.mean(batch_objective))

#     if log_metric_history:

#         # Set model in test mode
#         self.model.train(False)

#         # Evaluate model
#         with torch.no_grad():
#             self._evaluate_model(self.metric_history[split], x_train, y_train, x_val, y_val)

#         # Print progress
#         self._print_progress(split, epoch)

#     else:

#         # Print average objective from last epoch
#         self._print_objective(split, epoch)

# # Set model in test mode
# self.model.train(False)

# # Evaluate model
# with torch.no_grad():
#     self._evaluate_model(self.final_metric[split], x_train, y_train, x_val, y_val)





# Get train predictions
# mu_list = vogn_optimizer.get_mc_predictions(vogn_model.forward, 
#                                             inputs = X_val, 
#                                             mc_samples = train_params['eval_mc_samples'], 
#                                             ret_numpy=False)